為什麼這個老是報錯? 即便從別的可以正常執行的複製過來還是報錯?

結果是因為 

gemini = ChatGoogleGenerativeAI(model = "gemini-pro", verbose = True, temperature = 0.6, google_api_key = os.environ["GOOGLE_API_KEY"]),

# 最後面多了一個","造成的，切記！！！！！！！

In [2]:
import os
from crewai import Agent, Task, Crew, Process

In [3]:
import textwrap
from IPython.display import HTML, display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [5]:
# 設定4個 LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI
# crewai 使用 Ollama 的方法：windows 只需要打開 ubuntu 視窗即可，不需要執行 litellm；Mac 則甚至不需要啟動任何程式，只要有安裝好 Ollama 即可調用。
# 與 autogen 不同，autogen 要調用 Ollama 需要執行 litellm，多一道步驟，比較麻煩
gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0.7),
# Ollama 大模型，可以看要用哪一個
# ollama = Ollama(model="openhermes_assistant")  # 這是用 modelfile 建立的
# ollama 2024/03/12 最強大模型 nous-hermes2
ollama = Ollama(model="nous-hermes2")

gemini = ChatGoogleGenerativeAI(model="gemini-pro",
                             verbose = True,
                             temperature = 0.6,
                             google_api_key = os.environ["GOOGLE_API_KEY"]
                             )


d:\TOMO.Project\crewAI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# 設定工具 tools
# Make sure to Install duckduckgo-search for this example:
# !pip install -U duckduckgo-search
# 最簡易型的工具定義方式
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

# 從 tools 模組中引入需要的工具
from tools.browser_tools import BrowserTools
from tools.calculator_tools import CalculatorTools
from tools.file_tools import FileTools
from tools.search_tools import SearchTools
from tools.sec_tools import SECTools
# 下面添加 tools 時，必須用 類別.方法 加入工具
# BrowserTools.scrape_and_summarize_website
# CalculatorTools.calculate
# FileTools.write_file
# SearchTools.search_internet、SearchTools.search_news
# SECTools.search_10q、SECTools.search_10k
# 下面的是依樣畫葫蘆自行加入2行定義類別，親測可以匯入
from tools.find_papers_arxiv import FindPapersArxiv
from tools.get_top_hackernews_stories import GetTopHackernewsStories
# 然後就可以用下面的函式來使用工具，已測試可以~~~
# FindPapersArxiv.search_arxiv
# GetTopHackernewsStories.get_top_hackernews_stories

In [11]:
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting 
  actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_tool],
  debug_mode=True,
  # You can pass an optional llm attribute specifying what mode you wanna use.
  # It can be a local model through Ollama / LM Studio or a remote
  # model like OpenAI, Mistral, Antrophic or others (https://python.langchain.com/docs/integrations/llms/)
  #
  # Examples:
  #
  # from langchain_community.llms import Ollama
  # llm=gemini # was defined above in the file
  #
  # from langchain_openai import ChatOpenAI
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7)
)
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  debug_mode=True,
  # llm=gemini
  # (optional) llm=ollama_llm
)

In [12]:
# 上面建立 agent 時，可以不指定 llm，這裡才指定
researcher.llm = ollama
writer.llm = ollama

In [13]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your answer MUST be the full blog post of at least 5 paragraphs and finally using taditional chinese writing.""",
  agent=writer
)

ValidationError: 1 validation error for Task
expected_output
  Field required [type=missing, input_value=<unprintable dict object>, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

In [1]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

NameError: name 'Crew' is not defined

In [23]:
# Get your crew to work!
result = crew.kickoff()

# print("######################")
# print(result)
to_markdown(result)

[DEBUG]: Working Agent: Senior Research Analyst
[INFO]: Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report


> Entering new CrewAgentExecutor chain...
I tried to use a tool and give a final answer at the same time, I must choose only one.```
Use Tool: duckduckgo_search
Input: "latest advancements in AI in 2024"
``` 

In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of-the-art AI models ... Contributor. Nov 1, 2023,03:43am EDT. Share to Facebook. Share to Twitter. Share to Linkedin. The Top 5 Artificial Intelligence (AI) Trends For 2024. Adobe Stock. It's been a year since OpenAI ... This year's trends reflect a deepening sophistication and caution in AI development and deployment strategies, with 

> 在2024年，AI技术将继续发展并影响各种行业。多模式人工智能将成为更加普遍的事实，使人工智能系统能够同时处理不同类型的数据。量子计算也将贡献于开发更为复杂的人工智能算法。边缘AI将使AI应用更加可访问和节能。
> 
> 在医疗行业，人工智能将通过精准医学并提高患者结果来改善病患经历。金融行业将集成人工智能进 fraud detection系统, risk analysis和投资策略, 而自然语言处理（NLP）的前沿发展将提升客户服务聊天机器人、语音助手和情感分析工具。
> 
> 物流行业将使用AI优化供应链管理，路由和仓库操作，从而降低成本并增加效率。自动汽车也将更为普遍，改善交通和送货服务。能源领域将利用人工智能进行设施保养的预测，防止故障并提高生产率。
> 
> 总之，在2024年，人工智能的发展和部署将强调道德、安全和监管合规方面。随着这些技术成熟，其对各种行业的潜力影响将更加显著，驱动创新和增长。
> 
> 在2024年，重要的人工智能进展包括生成性AI成为非技术个体的更多有用，实际预期，多模态人工智能，较小的语言模型和开源前沿，GPU短缺和云成本，模型优化的可访问性，定制的本地模型和数据管道以及强大的虚拟代理，使用AI模型本地运行。这些发展将影响各种行业，包括医疗、金融、物流、交通和能源，并重新强调道德、安全和监管合规方面。